In [1]:
import os
import pandas as pd
import numpy as np
import fitsio
from pathlib import Path, PurePath
from dotenv import load_dotenv

load_dotenv()

True

In [117]:
def create_lookup_8nb(nx, ny):
    """ Pre-compute the 8-connectivity lookup table. This will be shared across parallel workers.
    :param nx:
    :param ny:
    :return:
    """
    # List of relative 2D coordinates for 8-neighbour connectiviy (9-element list). 1st one is the origin pixel.
    coords_8nb = np.array([[0, 0], [-1, 0], [-1, -1], [0, -1], [1, -1], [1, 0], [1, 1], [0, 1], [-1, 1]])
    # Array of 2D coordinates for a 4096 x 4096 array. Matrix convention is kept. [rows, cols] = [y-axis, x-axis]
    coords_1d = np.arange(nx * ny)
    coordy, coordx = np.unravel_index(coords_1d, [ny, nx]) # also possible by raveling a meshgrid() output
    coords2d = np.array([coordy, coordx])
    # Create the array of 2D coordinates of 8-neighbours associated with each pixel.
    # pixel 0 has 8 neighbour + itself, pixel 1 has 8 neighbour + itself, etc...
    coords2d_8nb = coords2d[np.newaxis, ...] + coords_8nb[..., np.newaxis]
    # Handle off-edges coordinates by clipping to the edges, operation done in-place. Here, square detector assumed.
    # to per-axis clipping if that ever changes for another instrument.
    np.clip(coords2d_8nb, 0, nx-1, out=coords2d_8nb)
    # Convert to 1D coordinates.
    lookup_coords = np.array([coords2d_8nb[i, 0, :] * nx + coords2d_8nb[i, 1, :] for i in range(len(coords_8nb))],
                         dtype='int32', order='C').T
    return lookup_coords


def extract_coincidentals(spikes_list, idx):
    # Spikes coordinates at given wavelength index
    spikes_w = spikes_list[idx]
    # Associated neighbour coordinates
    nb_pixels = index_8nb[spikes_w[0, :], :]
    # Sublist of spikes data that will excludes the one serving as template
    spikes_sublist = spikes_list[:idx] + spikes_list[idx + 1:]
    # Coincidental cross-referencing.
    # mask_w_arr = np.array([np.isin(nb_pixels, index_8nb[spikes[0, :], :]).any(axis=1) for spikes in spikes_sublist])
    mask_w_arr = np.array([np.isin(nb_pixels, spikes[0, :]).any(axis=1) for spikes in spikes_sublist])
    select_pixels = mask_w_arr.any(axis=0)
    coords_w = spikes_w[0, select_pixels]
    w_tables = np.insert(mask_w_arr[:, select_pixels], idx, True, axis=0)
    # Retrieve intensity values for the selected coordinates
    intensities = spikes_w[1:, select_pixels]
    arr_w = np.concatenate([coords_w[np.newaxis, ...], intensities, w_tables], axis=0)
    arr_w = np.insert(arr_w, 3, idx, axis=0)
    return arr_w


def process_group(group_n):
    fpaths = path_Series.loc[group_n]
    spikes_list = [fitsio.read(os.path.join(os.environ['SPIKESDATA'], f)) for f in fpaths]
    group_data = np.concatenate([extract_coincidentals(spikes_list, i) for i in range(7)], axis=1)
    column_names = ['coords', 'int1', 'int2', 'wref', 'w0', 'w1', 'w2', 'w3', 'w4', 'w5', 'w6']
    coincidental_spikes_df = pd.DataFrame(group_data.T, columns=column_names)
    coincidental_spikes_df['GroupNumber'] = group_n
    return coincidental_spikes_df

In [4]:
SPIKESDATA = os.environ['SPIKESDATA']
print(SPIKESDATA)
column_names = ['coords', 'int1', 'int2', 'wref', 'w1', 'w2', 'w3', 'w4', 'w4', 'w5', 'w6', 'w7']

/Users/rattie/data/spikes


In [30]:
index_8nb = create_lookup_8nb(4096, 4096)
print(index_8nb.shape)
print(index_8nb[0, :])

(16777216, 9)
[   0    0    0    0 4096 4096 4097    1    1]


In [8]:
spikes_df = pd.read_parquet(Path(os.environ['SPIKESDATA'], 'spikes_df_2018.parquet'),
                                engine='pyarrow')

,Path,Size,Time,Wavelength,GroupNumber
1,2018/01/01/2018-01-01T00h00m00.57Z_0094.spikes...,46080,2018-01-01 00:00:00.570000+00:00,94,1
2,2018/01/01/2018-01-01T00h00m02.07Z_0335.spikes...,72000,2018-01-01 00:00:02.070000+00:00,335,1
3,2018/01/01/2018-01-01T00h00m05.84Z_0193.spikes...,25920,2018-01-01 00:00:05.840000+00:00,193,1
4,2018/01/01/2018-01-01T00h00m06.58Z_0304.spikes...,77760,2018-01-01 00:00:06.580000+00:00,304,1
5,2018/01/01/2018-01-01T00h00m08.07Z_0131.spikes...,51840,2018-01-01 00:00:08.070000+00:00,131,1
...,...,...,...,...,...
50386,2018/01/01/2018-01-01T23h59m41.84Z_0193.spikes...,54720,2018-01-01 23:59:41.840000+00:00,193,7199
50387,2018/01/01/2018-01-01T23h59m42.58Z_0304.spikes...,178560,2018-01-01 23:59:42.580000+00:00,304,7199
50388,2018/01/01/2018-01-01T23h59m44.07Z_0131.spikes...,135360,2018-01-01 23:59:44.070000+00:00,131,7199
50389,2018/01/01/2018-01-01T23h59m46.35Z_0171.spikes...,48960,2018-01-01 23:59:46.350000+00:00,171,7199


In [9]:
spikes_df2 = spikes_df.set_index(['GroupNumber', 'Time'])
path_Series = spikes_df2['Path']
path_Series

GroupNumber  Time                            
1            2018-01-01 00:00:00.570000+00:00    2018/01/01/2018-01-01T00h00m00.57Z_0094.spikes...
             2018-01-01 00:00:02.070000+00:00    2018/01/01/2018-01-01T00h00m02.07Z_0335.spikes...
             2018-01-01 00:00:05.840000+00:00    2018/01/01/2018-01-01T00h00m05.84Z_0193.spikes...
             2018-01-01 00:00:06.580000+00:00    2018/01/01/2018-01-01T00h00m06.58Z_0304.spikes...
             2018-01-01 00:00:08.070000+00:00    2018/01/01/2018-01-01T00h00m08.07Z_0131.spikes...
                                                                       ...                        
7199         2018-01-01 23:59:41.840000+00:00    2018/01/01/2018-01-01T23h59m41.84Z_0193.spikes...
             2018-01-01 23:59:42.580000+00:00    2018/01/01/2018-01-01T23h59m42.58Z_0304.spikes...
             2018-01-01 23:59:44.070000+00:00    2018/01/01/2018-01-01T23h59m44.07Z_0131.spikes...
             2018-01-01 23:59:46.350000+00:00    2018/01/01/201

In [13]:
tintervals = pd.interval_range(start=pd.Timestamp('2018-01-01 00:00:00', tz='UTC'),
                                   end=pd.Timestamp('2018-01-01 23:59:59', tz='UTC'),
                                   freq='H', closed='left')

tint = tintervals[0]
print(tint.left, tint.right)

2018-01-01 00:00:00+00:00 2018-01-01 01:00:00+00:00


In [16]:
groups = spikes_df['GroupNumber'].loc[(spikes_df['Time'] >= tint.left) & (spikes_df['Time'] < tint.right)].unique()
print(groups)
group_n = groups[0]

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
 235 236 237 238 239 240 241 242 243 244 245 246 24

In [17]:
path_Series.loc[groups[0]]

Time
2018-01-01 00:00:00.570000+00:00    2018/01/01/2018-01-01T00h00m00.57Z_0094.spikes...
2018-01-01 00:00:02.070000+00:00    2018/01/01/2018-01-01T00h00m02.07Z_0335.spikes...
2018-01-01 00:00:05.840000+00:00    2018/01/01/2018-01-01T00h00m05.84Z_0193.spikes...
2018-01-01 00:00:06.580000+00:00    2018/01/01/2018-01-01T00h00m06.58Z_0304.spikes...
2018-01-01 00:00:08.070000+00:00    2018/01/01/2018-01-01T00h00m08.07Z_0131.spikes...
2018-01-01 00:00:10.350000+00:00    2018/01/01/2018-01-01T00h00m10.35Z_0171.spikes...
2018-01-01 00:00:11.080000+00:00    2018/01/01/2018-01-01T00h00m11.08Z_0211.spikes...
Name: Path, dtype: object

In [20]:
fpaths = path_Series.loc[groups[0]]
spikes_list = [fitsio.read(Path(SPIKESDATA, f).as_posix()) for f in fpaths]
spikes_list[0], spikes_list[1]

(array([[   16790,    18914,    23002, ..., 16705662, 16755371, 16763519],
        [     505,       16,       40, ...,        9,        9,       10],
        [       0,        0,        0, ...,        0,        0,       -1]],
       dtype=int32),
 array([[   10764,    11421,    18668, ..., 16743183, 16753046, 16764379],
        [     150,       72,      630, ...,        9,       13,        8],
        [       0,       -1,        0, ...,        0,        0,        0]],
       dtype=int32))

In [21]:
for spikes in spikes_list:
    print(spikes.shape)

(3, 11907)
(3, 20566)
(3, 4830)
(3, 21027)
(3, 13184)
(3, 4201)
(3, 9492)


In [99]:
idx = 0
# Spikes coordinates at given wavelength index
spikes_w = spikes_list[idx]
# Associated neighbour coordinates
nb_pixels = index_8nb[spikes_w[0, :], :]
print(nb_pixels.shape)

(11907, 9)


In [88]:
index_8nb[11000, :]

array([11000,  6904,  6903, 10999, 15095, 15096, 15097, 11001,  6905],
      dtype=int32)

In [100]:
# Sublist of spikes data that will excludes the one serving as template
spikes_sublist = spikes_list[:idx] + spikes_list[idx + 1:]
# Coincidental cross-referencing.
mask_w_arr = np.array([np.isin(nb_pixels, spikes[0, :]).any(axis=1) for spikes in spikes_sublist])
select_pixels = mask_w_arr.any(axis=0)
coords_w = spikes_w[0, select_pixels]
w_tables = np.insert(mask_w_arr[:, select_pixels], idx, True, axis=0)
# Retrieve intensity values for the selected coordinates
intensities = spikes_w[1:, select_pixels]
arr_w = np.concatenate([coords_w[np.newaxis, ...], intensities, w_tables], axis=0)
arr_w = np.insert(arr_w, 3, idx, axis=0)

In [101]:
print(mask_w_arr.shape)
print(mask_w_arr)

(6, 11907)
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]


In [102]:
print(select_pixels.shape)
print(select_pixels)

(11907,)
[False False False ... False False False]


In [103]:
print(coords_w.shape)

(350,)


In [108]:
print(mask_w_arr[:, select_pixels])

[[False False False ... False False False]
 [ True False False ... False False False]
 [False False False ... False  True False]
 [False False False ...  True False  True]
 [False False False ... False False False]
 [False  True  True ... False False False]]


In [104]:
print(w_tables.shape)
print(w_tables)

(7, 350)
[[ True  True  True ...  True  True  True]
 [False False False ... False False False]
 [ True False False ... False False False]
 ...
 [False False False ...  True False  True]
 [False False False ... False False False]
 [False  True  True ... False False False]]


In [107]:
print(intensities.shape)
print(arr_w.shape)
print(arr_w)

(2, 350)
(11, 350)
[[  149558   205896   209992 ... 13290470 14044037 14279287]
 [     117      165      111 ...       19       15        9]
 [       0        0        0 ...        3        4        0]
 ...
 [       0        0        0 ...        1        0        1]
 [       0        0        0 ...        0        0        0]
 [       0        1        1 ...        0        0        0]]


In [118]:
group_data = np.concatenate([extract_coincidentals(spikes_list, i) for i in range(7)], axis=1)

In [120]:
df = process_group(1)

In [122]:
df

,coords,int1,int2,wref,w0,w1,w2,w3,w4,w5,w6,GroupNumber
0,149558,117,0,0,1,0,1,0,0,0,0,1
1,205896,165,0,0,1,0,0,0,0,0,1,1
2,209992,111,0,0,1,0,0,0,0,0,1,1
3,239694,151,1,0,1,1,0,0,0,0,0,1
4,348983,102,0,0,1,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2181,7735529,75,35,6,0,0,0,1,0,0,1,1
2182,14984731,48,21,6,0,0,0,1,0,0,1,1
2183,15568514,12,1,6,0,0,0,0,1,0,1,1
2184,16041995,13,3,6,1,0,0,0,0,0,1,1
